In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import rushd as rd
import scipy as sp
import seaborn as sns

from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
# Setup data loading
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'
data_list = [base_path/'2024.11.18_exp117.5'/'export'/'plate1', base_path/'2024.11.18_exp117.5'/'export'/'plate2',]
plates = pd.DataFrame({
    'data_path': data_list,
    'yaml_path': [p/'wells.yaml' for p in data_list],
    'exp': ['exp117.5']*2
})
output_path = rd.rootdir/'output'/'lenti_therapeutic'
cache_path = output_path/'lenti_therapeutic.gzip'

for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)

In [ ]:
# Load data
data = pd.DataFrame()
channel_list = ['mRuby2-A','mGL-A']

if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)
    for c in channel_list: data = data[data[c]>0]
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A',]
for channel in channel_list:
    gates[channel] = data[data['construct']=='UI'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

display(gates)

In [ ]:
data['marker'] = data['mGL-A']
data.loc[data['name'].str.contains('FMRP'), 'marker'] = data.loc[data['name'].str.contains('FMRP'), 'mRuby2-A']
data['output'] = data['mRuby2-A']
data.loc[data['name'].str.contains('FMRP'), 'output'] = data.loc[data['name'].str.contains('FMRP'), 'mGL-A']

In [ ]:
x = 'mGL-A'
y = 'mRuby2-A'
plot_df = data[(data['dox']==1000) & (data['construct']!='UI')].groupby(['biorep','construct']).sample(1000)
g = sns.displot(data=plot_df, x=x, y=y, hue='construct', palette=main_palette, kind='kde',
                row='biorep', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False, levels=8)

for (biorep, construct), ax in g.axes_dict.items():
    ax.axvline(gates[x].values[0], c='black', ls=':', zorder=0)
    ax.axvline(2e3, c='black', ls=':', zorder=0)
    ax.axhline(gates[y].values[0], c='black', ls=':', zorder=0)

In [ ]:
# Manually adjust mGL gate
gates['mGL-A'] = [2e3]

# Gate data by marker expression
def gate_data(df, gates):
    df = df.copy()
    exp = df['exp'].values[0] # the same for entire df, assuming df = data.groupby('exp')
    gates_dict = gates.set_index('exp').to_dict('dict') # format: column -> {index: value}
    df['expressing'] = df['marker'] > gates_dict['mGL-A'][exp]
    df.loc[df['name'].str.contains('FMRP'), 'expressing'] = df.loc[df['name'].str.contains('FMRP'), 'marker'] > gates_dict['mRuby2-A'][exp]
    return df

data = data.groupby('exp')[data.columns].apply(lambda x: gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
df = data[(data['expressing']) & (data['construct']!='UI')]

In [ ]:
x = 'mGL-A'
y = 'mRuby2-A'
plot_df = df[(df['dox']==1000)].groupby(['biorep','construct']).sample(1000)
g = sns.displot(data=plot_df, x=x, y=y, hue='construct', palette=main_palette, kind='kde',
                row='biorep', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False, levels=8)

for (biorep, construct), ax in g.axes_dict.items():
    ax.axvline(gates[x].values[0], c='black', ls=':', zorder=0)
    ax.axhline(gates[y].values[0], c='black', ls=':', zorder=0)

In [ ]:
# Bin data and calculate statistics
df_quantiles, stats, _, fits = base.calculate_bins_stats(df, stat_list=[sp.stats.gmean, np.std, sp.stats.variation], by=['construct','dox','biorep'])
stats = stats.merge(metadata, how='left', on='construct')
fits = fits.merge(metadata, how='left', on='construct')

In [ ]:
ts_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}
marker_list = ['o', 'v', 'D', 'X']

In [ ]:
display(axes.flatten())

In [ ]:
fig, axes = plt.subplots(2,4, figsize=(12,8), gridspec_kw=dict(wspace=0.5, hspace=0.5))

for i in range(2):
    if i: 
        plot_df = stats[(stats['dox']==1000) & (stats['name'].str.contains('FMRP'))]
        plot_df2 = fits[(fits['dox']==1000) & (fits['name'].str.contains('FMRP'))]
    else:
        plot_df = stats[(stats['dox']==1000) & (stats['name'].str.contains('FXN'))]
        plot_df2 = fits[(fits['dox']==1000) & (fits['name'].str.contains('FXN'))]

    ax = axes[i,0]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='ts_kind', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=8, edgecolor='white', linewidth=1)
    ax.set(title='Mean', xlabel='', ylabel='', yscale='log', )#ylim=(1e3,2e4),)

    ax = axes[i,1]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='ts_kind', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=8, edgecolor='white', linewidth=1)
    ax.set(title='Std.', xlabel='', ylabel='', yscale='log', )#ylim=(1e3,2e4),)

    ax = axes[i,2]
    for construct, group in plot_df2.groupby('construct'):
        sns.stripplot(data=group, x='ts_kind', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=8, edgecolor='white', linewidth=1)
    ax.set(title='Slope', xlabel='', ylabel='',)

    ax = axes[i,3]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='ts_kind', y='output_variation', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=8, edgecolor='white', linewidth=1)
    ax.set(title='CV', xlabel='', ylabel='',)

for ax in axes.flatten():
    ax.set_xticklabels([ts_label[x.get_text()] for x in ax.get_xticklabels()], rotation=45, ha='right',)
    sns.despine(ax=ax)

fig.savefig(rd.outfile(output_path/'stats.png'))